In [ ]:
import json
from openai import OpenAI
import os
from pathlib import Path

In [ ]:
PROMPT_TEMPLATE = """
You are an experienced IELTS speaking examiner, expanding the IELTS speaking topic "{core_topic['name']}".  

### Topic Overview:  
{core_topic['description']}  

### Reference Subtopics:  
{json.dumps(core_topic['sub_topics'], ensure_ascii=False, indent=2)}  

### Your Task:  
Generate 10 specific expanded subtopics based on the core theme "{core_topic['name']}", following these guidelines:  

1. Real-life Contexts: Consider how this topic applies to different settings (e.g., home, school, workplace, social interactions).  
2. Cultural Perspectives: Compare attitudes, habits, and traditions across different cultures or countries.  
3. Personalized Angles: Include aspects related to personal experiences, future possibilities, or changes over time.  
4. IELTS Relevance: Ensure that each expanded topic allows test-takers to develop detailed responses.  

### Output Format:  
Core Theme: {core_topic['name']}  
- Expanded Topic 1: [Topic Name] – [Short Explanation]  
- Expanded Topic 2: [Topic Name] – [Short Explanation]  
- Expanded Topic 3: [Topic Name] – [Short Explanation]  
- Expanded Topic 4: [Topic Name] – [Short Explanation]  
- Expanded Topic 5: [Topic Name] – [Short Explanation]  
...  

### Example Output:  
Core Theme: Travel  
- Solo vs. Group Travel – Discuss advantages and challenges of traveling alone versus with others.  
- Travel and Technology – How mobile apps and digital platforms have changed the way people travel.  
- Eco-friendly Tourism – The impact of sustainable travel and how individuals can reduce their carbon footprint.  
- Cultural Sensitivity in Travel – How travelers should respect and adapt to different cultures.  
- The Future of Space Tourism – Would you travel to space if given the chance? Why or why not?  

Ensure that the expanded topics are engaging, diverse, and practical for IELTS test preparation.  
"""

In [ ]:
def expand_topic_with_llm(theme_data, client):
    prompt = PROMPT_TEMPLATE.format(
        theme_name=theme_data["theme_name"],
        reasoning=theme_data["reasoning"],
        subtopics="\n".join(f"- {t}" for t in theme_data["subtopics"])
    )
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are an experienced IELTS speaking examiner"},
            {"role": "user", "content": prompt}
        ],
        temperature=1.3,
        stream=False
    )
    
    result = response.choices[0].message.content
    return result;

In [ ]:
def save_expanded_topics(theme_name, content, output_dir):
    # 确保目录存在
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 生成文件名
    safe_theme_name = re.sub(r'[^\w\u4e00-\u9fff-]', '_', theme_name)
    output_path = output_dir / f"{safe_theme_name}.txt"
    
    # 保存文件
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"主题 '{theme_name}' 的扩展已保存到: {output_path}")

In [ ]:
# 从环境变量获取API密钥
api_key = os.getenv('DEEPSEEK_API_KEY')
if not api_key:
    raise ValueError("请在.env文件中设置DEEPSEEK_API_KEY环境变量")

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

In [ ]:
# 读取summary.json
summary_path = Path("../Extraction/Part1/summary/summary.json")
with open(summary_path, 'r', encoding='utf-8') as f:
    summary_data = json.load(f)

In [ ]:
# 输出目录
output_dir = "expanded_topics"

# 逐个处理核心主题
for theme in summary_data["core_themes"]:
    try:
        # 生成扩展主题
        expanded_content = expand_topic_with_llm(theme, client)
        
        # 保存结果
        save_expanded_topics(theme["theme_name"], expanded_content, output_dir)
    except Exception as e:
        print(f"处理主题 '{theme['theme_name']}' 时出错: {str(e)}")